## Data Loader
#### Script to load the indicator data to a SQL Server Database

In [1]:
# Imports
import pandas as pd # Dataframe
from pyjstat import pyjstat
import json # Parsing json to object
import requests # Making HTTP get requests
import pycountry
import pyodbc
import math
import numpy as np

In [2]:
# Create Lists
# Commonwealth Countries
commonwealth_countries = [
    "Botswana", "Cameroon", "Gabon", "Gambia", "Ghana", "Kenya",
    "Eswatini", "Lesotho", "Malawi", "Mauritius", "Mozambique",
    "Namibia", "Nigeria", "Rwanda", "Seychelles", "Sierra Leone",
    "South Africa", "Togo", "Uganda", "Tanzania, United Republic of", "Zambia",
    "Bangladesh", "Brunei Darussalam", "India", "Malaysia", "Maldives",
    "Pakistan", "Singapore", "Sri Lanka", "Antigua and Barbuda", "Bahamas",
    "Barbados", "Belize", "Canada", "Dominica", "Grenada", "Guyana",
    "Jamaica", "Saint Lucia", "Saint Kitts and Nevis", "Saint Vincent and The Grenadines",
    "Trinidad and Tobago", "Cyprus", "Malta", "United Kingdom", "Australia",
    "Fiji", "Kiribati", "Nauru", "New Zealand", "Papua New Guinea", "Samoa",
    "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu"
]

# Gender Indicators
gender_indicators = [
    "FIN21.T.D.2017.1","FIN21.T.D.2017.2","FIN21.T.D.2017","SG.GEN.PARL.ZS",
    "SG.GEN.MNST.ZS","SE.SEC.ENRR.FE","UIS.FGP.5T8.F600","SL.TLF.CACT.FE.ZS",
    "SG.LAW.NODC.HR","SG.OWN.LDAL.FE.ZS","SG.OPN.BANK.EQ","SG.CNT.SIGN.EQ",
    "SP.DYN.SMAM.FE","SP.DYN.SMAM.MA","SP.M15.2024.FE.ZS","SP.M18.2024.FE.ZS",
    "SG.VAW.1549.ME.ZS","SG.VAW.15PL.ME.ZS","SG.VAW.1549.LT.ME.ZS","SG.VAW.15PL.LT.ME.ZS",
    "SG.LEG.DVAW","SH.STA.MMRT","SH.STA.MMRT.NE","SP.DYN.LE00.FE.IN","SP.DYN.LE00.MA.IN","SP.DYN.LE00.IN"
]

# Get 3 digit ISO codes
country_iso_codes = {}
for country in commonwealth_countries:
    try:
        iso_code = pycountry.countries.get(name=country).alpha_3 # Trinidad and Tobago | Trinidad & Tobago | trinidad and tobago [TTO] - https://www.iban.com/country-codes
        country_iso_codes[country] = iso_code
    except AttributeError:
        print(f"ISO code not found for {country}")

In [3]:
def download_indicators(country_list, indicator_list):
    isocode_filter = []
    indicator_filter = []
    for country in country_list:
        iso_code = country_iso_codes[country]
        isocode_filter.append(iso_code)
    for indicator in indicator_list:
        indicator_filter.append(indicator) 
    # Leverage on the parameter structure of the API
    api_url = f'http://api.worldbank.org/v2/country/{";".join(isocode_filter)}/indicator/{";".join(indicator_filter)}?format=jsonstat&gapfill=N&source=14'
    dataset = pyjstat.Dataset.read(api_url)
    df = dataset.write('dataframe')
    return df

In [4]:
# Gets the sql server connection string
def get_conn_string():
    server = '.\SQLEXPRESS' 
    database = 'Gender_Declaration' 
    username = '' 
    password = ''
    if (username != ''):
        return 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    else: # Trusted Connection
        return 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;'

In [5]:
# Inserts the dataframe rows to SQL Server Table
def df_to_table(table_name, df):
    cnxn = pyodbc.connect(get_conn_string())
    cursor = cnxn.cursor()
    counter = 0;
    print(f'Processing Table: { table_name }')
    try:
        for index, row in df.iterrows():
            cursor.execute(f"INSERT INTO {table_name} ({','.join(df.columns)}) values({','.join('?' * len(df.columns))})", row.values.tolist())
            counter = counter + 1
            if (counter >= 500):
                print('Added 500 rows ...')
                counter = 0
        cnxn.commit()
    finally:
        cursor.close()

In [6]:
# Clear any existing data in the database
def empty_database():
    table_names = ['FACT_INDICATOR_VALUE', 'DIM_INDICATOR', 'DIM_COUNTRY', 'DIM_REGION']
    cnxn = pyodbc.connect(get_conn_string())
    cursor = cnxn.cursor()
    try:
        for table_name in table_names:
            print(f'Clearing TABLE {table_name}')
            cursor.execute(f'DELETE FROM {table_name}')
        cnxn.commit()
    finally:
        cursor.close()

In [7]:
# Insert lookup data

empty_database()

# REGIONS
df = pd.DataFrame(columns=['ID', 'NAME'])
df.loc[0] = { 'ID': 1, 'NAME': 'Commonwealth'}
df_to_table('DIM_REGION', df)

# COUNTRIES
index = 0
df = pd.DataFrame(columns=['ID', 'NAME', 'ISO_CODE', 'REGION_ID'])
for country in commonwealth_countries:
    df.loc[index] = { 'ID': index + 1, 'NAME': country, 'ISO_CODE': country_iso_codes[country], 'REGION_ID': 1}
    index = index + 1
df_to_table('DIM_COUNTRY', df)

# INDICATORS
index = 0
df = pd.DataFrame(columns=['ID', 'TITLE', 'CODE'])
for indicator in gender_indicators:
    dfx = download_indicators(['Botswana'], [indicator])
    df.loc[index] = { 'ID': index + 1, 'TITLE': dfx['Series'].unique()[0], 'CODE': indicator }
    index = index + 1
df_to_table('DIM_INDICATOR', df)

Clearing TABLE FACT_INDICATOR_VALUE
Clearing TABLE DIM_INDICATOR
Clearing TABLE DIM_COUNTRY
Clearing TABLE DIM_REGION
Processing Table: DIM_REGION
Processing Table: DIM_COUNTRY
Processing Table: DIM_INDICATOR


In [8]:
# Load indicators data to database
country_id = 0
for country in commonwealth_countries:
    print(f'Loading data for: {country}')
    df = pd.DataFrame(columns=['INDICATOR_ID', 'COUNTRY_ID', 'VALUE', 'YEAR'])
    gindex = 0
    country_id = country_id + 1
    indicator_id = 0
    for indicator in gender_indicators:
        indicator_id = indicator_id + 1
        dfx = download_indicators([country], [indicator])
        for index, row in dfx.iterrows():
            value = row['value']
            if (value == None or math.isnan(value)):
                value = None
            df.loc[gindex] = { 'INDICATOR_ID': indicator_id, 'COUNTRY_ID': country_id, 'VALUE': value , 'YEAR': row['Year'] }
            gindex = gindex + 1
    df = df.replace({np.nan: None})
    df_to_table('FACT_INDICATOR_VALUE', df)

Loading data for: Botswana
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Cameroon
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Gabon
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Gambia
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Ghana
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Kenya
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Eswatini
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Lesotho
Processing Table: FACT_INDICATOR_VALUE
Added 500 rows ...
Added 500 rows ...
Added 500 rows ...
Loading data for: Malawi
Pro